# Getting the Data 

In [1]:
import facebook

In [2]:

token = '465406930468068|8e5e0af18cdd664fb0f0126686b3c821'

graph = facebook.GraphAPI(access_token=token, version='2.7')


In [3]:
graph_dict = graph.get_connections('24369314439', connection_name='posts')

In [4]:
graph_dict.keys()

[u'paging', u'data']

In [5]:
data = graph_dict['data']

In [6]:
import pandas as pd
df = pd.DataFrame(data)

## via requests 

In [77]:
import json

In [75]:
import requests

page_id = '24369314439'
token = '465406930468068|8e5e0af18cdd664fb0f0126686b3c821'

r = requests.get('https://graph.facebook.com/v2.9/' + page_id + '/posts?access_token=' + token + '&limit=100')

In [79]:
data= r.json()['data']

In [80]:
df = pd.DataFrame(data)

The data as a table.

In [81]:
df.head()

,created_time,id,message,story
0,2017-06-12T11:50:24+0000,24369314439_10154944511439440,Germany Decides: On the road to the elections\...,DW News updated their cover photo.
1,2017-06-12T11:09:00+0000,24369314439_10154944432029440,"Six cities, two reporters, one question: What ...",NaN
2,2017-06-12T09:46:14+0000,24369314439_10154944302089440,UPDATE: Russian opposition leader Alexei Naval...,NaN
3,2017-06-12T08:39:39+0000,24369314439_10154944182269440,For all of our coverage in the run-up to the G...,NaN
4,2017-06-12T07:41:03+0000,24369314439_10154944052244440,"Cheering, hundreds entered the EU in what Pres...",NaN


In [82]:
df.shape

(100, 4)

In [83]:
df['time'] = pd.to_datetime(df.created_time)

In [84]:
df.time

0    2017-06-12 11:50:24
1    2017-06-12 11:09:00
2    2017-06-12 09:46:14
3    2017-06-12 08:39:39
4    2017-06-12 07:41:03
5    2017-06-12 06:39:55
6    2017-06-12 05:47:15
7    2017-06-12 04:47:00
8    2017-06-12 04:32:28
9    2017-06-12 04:03:00
10   2017-06-12 02:03:00
11   2017-06-12 00:03:00
12   2017-06-11 22:03:00
13   2017-06-11 20:03:00
14   2017-06-11 18:03:00
15   2017-06-11 15:53:04
16   2017-06-11 15:09:07
17   2017-06-11 14:23:31
18   2017-06-11 13:21:48
19   2017-06-11 12:23:12
20   2017-06-11 11:38:34
21   2017-06-11 10:47:57
22   2017-06-11 09:49:59
23   2017-06-11 09:03:00
24   2017-06-11 07:02:00
25   2017-06-11 05:03:00
26   2017-06-11 03:01:00
27   2017-06-11 01:02:00
28   2017-06-10 23:02:00
29   2017-06-10 21:03:00
             ...        
70   2017-06-08 17:19:21
71   2017-06-08 14:01:41
72   2017-06-08 13:03:37
73   2017-06-08 11:51:51
74   2017-06-08 10:59:42
75   2017-06-08 10:57:08
76   2017-06-08 09:58:56
77   2017-06-08 08:54:43
78   2017-06-08 06:01:01


# Named Entity Recognition 

In [10]:
import spacy
nlp = spacy.load('en')


In [29]:
import re

In [85]:
# clean the text

def preprocess_text(text):
    out = text.replace(u"'s", u"")
    out = out.replace(u"-", u" ")
    return out

In [86]:
# extract named entities

def get_named_entities(text):
    doc = nlp(text)
    out = [{'label':ent.label_, 'text':ent.text} for ent in doc.ents]
    return out

In [87]:
# apply functions from above

df['cleaned_text'] = df.message.apply(preprocess_text)
df['named_entities'] = df.cleaned_text.apply(get_named_entities)

In [88]:
df.message.iloc[3]

u'For all of our coverage in the run-up to the German election visit www.dw.com/germanelections and follow the hashtag #GermanyDecides.'

In [89]:
df.cleaned_text[3]

u'For all of our coverage in the run up to the German election visit www.dw.com/germanelections and follow the hashtag #GermanyDecides.'

In [97]:
df.named_entities[0]

[{'label': u'CARDINAL', 'text': u'\n\n'},
 {'label': u'CARDINAL', 'text': u'Six'},
 {'label': u'CARDINAL', 'text': u'two'},
 {'label': u'CARDINAL', 'text': u'one'},
 {'label': u'NORP', 'text': u'Germans'},
 {'label': u'DATE', 'text': u'2017'},
 {'label': u'PERSON', 'text': u'Sumi Somaskanda'},
 {'label': u'PERSON', 'text': u'Nina Haase'},
 {'label': u'DATE', 'text': u'this year'},
 {'label': u'ORDINAL', 'text': u'First'},
 {'label': u'GPE', 'text': u'Dresden'},
 {'label': u'ORG', 'text': u'DW News Facebook'},
 {'label': u'ORG', 'text': u'Twitter'}]

Get Persons

In [91]:
df['Persons'] = df.named_entities.apply(lambda x: [ne['text'] for ne in x if ne['label'] == 'PERSON'])

In [92]:
df.Persons

0       [Sumi Somaskanda, Nina Haase]
1       [Sumi Somaskanda, Nina Haase]
2                    [Alexei Navalny]
3                                  []
4                       [@Poroshenko]
5                           [Theresa]
6                                  []
7                                  []
8                   [Emmanuel Macron]
9                                  []
10                                 []
11                    [Gero Schliess]
12                                 []
13                                 []
14                                 []
15                    [Angela Merkel]
16                                 []
17                  [Donald J. Trump]
18                       [Poroshenko]
19                                 []
20                                 []
21                                 []
22                     [Erna Solberg]
23                                 []
24                                 []
25                                 []
26          

In [98]:
from geotext import GeoText
import pycountry

In [95]:
places = GeoText(df.cleaned_text[0])
print places.cities
# "London"

print GeoText(df.cleaned_text[0]).country_mentions


[u'Dresden']
OrderedDict([('DE', 2)])


In [96]:
df.cleaned_text[0]

u"Germany Decides: On the road to the elections\n\nSix cities, two reporters, one question: What matters most to Germans ahead of the 2017 federal elections? DW reporters Sumi Somaskanda and Nina Haase are hitting the road to find out. Each week they'll visit a different city or region that represents an important facet of this year big election issues. First stop is Dresden. Follow them on DW News Facebook or under the Twitter hashtag #GermanyDecides to keep up with each step on the road to the elections."

In [71]:
df.cleaned_text[3]

u'UK PM Theresa May plan to recruit Northern Ireland backers to stay in power has raised concerns in Dublin. Her cabinet reshuffle may signal a more open stance on Brexit talks with Europe.'

In [143]:
df['cities'] = df.cleaned_text.apply(lambda x: GeoText(x).cities)
df['countries'] = df.cleaned_text.apply(lambda x: dict(GeoText(x).country_mentions))

In [144]:
df.countries[20]

{'XK': 1}

In [142]:
def geotext_to_alpha_3(gt_dict):
    '''
    gt_od: OrderedDict from GeoText
    
    returns: dict including alpha_3 kuerzel
    '''
    out = {}
    for k in gt_dict.keys():
        country = pycountry.countries.get(alpha_2=k)
        new_key = country.alpha_3
        out[new_key] = gt_dict.pop(k)
    return out

In [141]:
# df['countries'] = df.countries.apply(geotext_to_alpha_3)

KeyError: 'XK'

In [147]:
pd.options.display.max_colwidth=200

In [150]:
df[['cleaned_text', 'named_entities']].head(30)

,cleaned_text,named_entities
0,"Germany Decides: On the road to the elections\n\nSix cities, two reporters, one question: What matters most to Germans ahead of the 2017 federal elections? DW reporters Sumi Somaskanda and Nina Ha...","[{u'text': u' ', u'label': u'CARDINAL'}, {u'text': u'Six', u'label': u'CARDINAL'}, {u'text': u'two', u'label': u'CARDINAL'}, {u'text': u'one', u'label': u'CARDINAL'}, {u'text': u'Germans', u'labe..."
1,"Six cities, two reporters, one question: What matters most to Germans ahead of the 2017 federal elections? Sumi Somaskanda and Nina Haase are hitting the road to find out. Tune in to our Facebook ...","[{u'text': u'Six', u'label': u'CARDINAL'}, {u'text': u'two', u'label': u'CARDINAL'}, {u'text': u'one', u'label': u'CARDINAL'}, {u'text': u'Germans', u'label': u'NORP'}, {u'text': u'2017', u'label'..."
2,UPDATE: Russian opposition leader Alexei Navalny arrested ahead of anti Kremlin protests. Supporters of Putin critic Navalny are expected to mark Monday public holiday with over 200 rallies nation...,"[{u'text': u'Russian', u'label': u'NORP'}, {u'text': u'Alexei Navalny', u'label': u'PERSON'}, {u'text': u'Kremlin', u'label': u'ORG'}, {u'text': u'Putin', u'label': u'GPE'}, {u'text': u'Navalny', ..."
3,For all of our coverage in the run up to the German election visit www.dw.com/germanelections and follow the hashtag #GermanyDecides.,"[{u'text': u'German', u'label': u'NORP'}, {u'text': u'GermanyDecides', u'label': u'ORG'}]"
4,"Cheering, hundreds entered the EU in what President @Poroshenko called a symbolic ""final breakaway"" from Moscow.","[{u'text': u'hundreds', u'label': u'CARDINAL'}, {u'text': u'EU', u'label': u'ORG'}, {u'text': u'@Poroshenko', u'label': u'PERSON'}, {u'text': u'Moscow', u'label': u'GPE'}]"
5,UK PM Theresa May plan to recruit Northern Ireland backers to stay in power has raised concerns in Dublin. Her cabinet reshuffle may signal a more open stance on Brexit talks with Europe.,"[{u'text': u'UK', u'label': u'GPE'}, {u'text': u'Theresa', u'label': u'PERSON'}, {u'text': u'Northern Ireland', u'label': u'LOC'}, {u'text': u'Dublin', u'label': u'GPE'}, {u'text': u'Brexit', u'la..."
6,The first bicycle ride took place 200 years ago.\n\nAny idea where? 🚲,"[{u'text': u'first', u'label': u'ORDINAL'}, {u'text': u'200 years ago', u'label': u'DATE'}]"
7,Foreigners released from prison in Peru aren't always free to leave the country; it takes them more than six months to get their exit papers and leaves them prone to more problems.,"[{u'text': u'Peru', u'label': u'GPE'}, {u'text': u'more than six months', u'label': u'DATE'}]"
8,Preliminary results have shown French President Emmanuel Macron political startup well ahead of traditional parties with over 30 percent in the first round of voting. The center right Republicans ...,"[{u'text': u'French', u'label': u'NORP'}, {u'text': u'Emmanuel Macron', u'label': u'PERSON'}, {u'text': u'over 30 percent', u'label': u'PERCENT'}, {u'text': u'first', u'label': u'ORDINAL'}, {u'tex..."
9,June 12 marks the World Day Against Child Labour.,"[{u'text': u'June 12', u'label': u'DATE'}, {u'text': u'the World Day Against', u'label': u'ORG'}]"


In [145]:
df.tail()

,created_time,id,message,story,time,cleaned_text,named_entities,Persons,cities,countries,countries_3
95,2017-06-07T09:50:46+0000,24369314439_10154927706699440,This is what the refugee crisis looks like thr...,NaN,2017-06-07 09:50:46,This is what the refugee crisis looks like thr...,[],[],[],{},{}
96,2017-06-07T08:50:16+0000,24369314439_10154927584724440,The base could be moved from Turkey to Jordan.,NaN,2017-06-07 08:50:16,The base could be moved from Turkey to Jordan.,"[{u'text': u'Turkey', u'label': u'GPE'}, {u'te...",[],[],"{u'JO': 1, u'TR': 1}",{u'DEU': 1}
97,2017-06-07T07:50:23+0000,24369314439_10154927366039440,Think the diplomatic crisis between Qatar and ...,NaN,2017-06-07 07:50:23,Think the diplomatic crisis between Qatar and ...,"[{u'text': u'Qatar', u'label': u'GPE'}, {u'tex...",[],[],{u'QA': 1},{u'DEU': 1}
98,2017-06-07T06:47:04+0000,24369314439_10154927267849440,BREAKING,NaN,2017-06-07 06:47:04,BREAKING,[],[],[],{},{}
99,2017-06-07T05:50:59+0000,24369314439_10154927167894440,"The Taliban it will ""face consequences"" if it ...",NaN,2017-06-07 05:50:59,"The Taliban it will ""face consequences"" if it ...","[{u'text': u'Taliban', u'label': u'ORG'}, {u't...",[Ashraf Ghani],[],{},{}


In [120]:
out = {}
for dc in df.countries_3:
    for k, v in dc.iteritems():
        if k in out.keys():
            out[k] = out[k] + v
        else:
            out[k] = v

In [121]:
out

{u'DEU': 50}